## Podstawy bezpieczeństwa komputerowego

# Notatka z pierwszego spotkania

## Otwieranie strony

To, że przeglądarka nie może (nie chce, nie potrafi) załadować jakieś strony to nic nie mówi o przyczynie.

Sprawdzanie serwisu przez przeglądarkę nie ma sensu, gdyż zbyt wiele rzeczy może się nie udać i nie wiadomo co faktycznie poszło nie tak.

Co się dzieje lepiej widać z poziomu [`wget`](https://www.gnu.org/software/wget/manual/html_node/index.html) lub [`curl`](https://curl.se/docs/manual.html).

```
# wget 87.205.94.150:80
--2021-03-10 14:01:00--  http://87.205.94.150/
Connecting to 87.205.94.150:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://dziurawiec/ [following]
--2021-03-10 14:01:00--  http://dziurawiec/
Resolving dziurawiec... failed: Name or service not known.
wget: unable to resolve host address “dziurawiec”
```

Wyraźnie widać, że serwer jest osiągalny:
```
Connecting to 87.205.94.150:80... connected.
```

Wyraźnie widać, że odpowidział:
```
HTTP request sent, awaiting response... 301 Moved Permanently
```

Ten [kod HTTP](https://developer.mozilla.org/en-US/docs/Web/HTTP/Status) to żadne zaskoczenie.
W przypadku [współdzielonego hostingu](https://en.wikipedia.org/wiki/Shared_web_hosting_service#Name-based) to dość oczywista sprawa, bo bez nazwy domenowej serwer nie wie, którego [vhost-a](http://httpd.apache.org/docs/current/vhosts/) ma użyć.
Równie często będzie to redirect z powodu użycia http, a nie https.

Nie udało się [rozwiązać nazwy](https://en.wikipedia.org/wiki/Domain_Name_System#DNS_resolvers), ale przynajmniej wiadomo gdzie problem.
```
Resolving dziurawiec... failed: Name or service not known.
wget: unable to resolve host address “dziurawiec”
```

Dziwny ten konfig bo nie jest to [domena w pełni kawalifikowana](https://en.wikipedia.org/wiki/Fully_qualified_domain_name), więc w różnych miejscach może się różnie rozwiązywać.

Z drugiej strony nie jest, aż tak zaskakujące. Może zasób ma być w założeniu dostępny "w siedzibie" i wtedy lokalny [proxy dns](https://en.wikipedia.org/wiki/Dnsmasq) (np. taki jak w domowym tzw. rutrze) by to rozwiązał.
Więc może wystarczy zmodyfikować plik [`hosts`](https://en.wikipedia.org/wiki/Hosts_(file)) dodająć:
```
87.205.94.150 dziurawiec
```

### Wnioski

Wydałoby:
* wiedzieć na czym polega http i dns,
* mieć konsolowe środowisko do debugu, tworzenia lub wykorzystywania dziwnych sytuacji,
* znać swój ulubiony OS.

## Rozglądanie się i domniemania

Do ustalenia:
* Czym jest ta strona?
  * statyczny html, a może jakiś famework, jakiś cms?
    * zamknięty/otwarty?
    * która wersja?
    * w czym napisany?
* Na czym jest ta strona?
  * jaki webserwer?
* OS hosta?

Czasami widać to np. w kodzie strony, np.:
```
<meta name="generator" content="WordPress 5.7" />
```

Czasami w ścieżkach do assetów, np.:
```
<script type='text/javascript' src='/wp-includes/js/comment-reply.min.js?ver=5.7' id='comment-reply-js'></script>
```

Czasami wystarczy coś doczytać o zależnościach, [np.](https://wordpress.org/support/article/requirements/):

```
To run WordPress we recommend your host supports the following:

    PHP version 7.4+
    MySQL version 5.6+ or MariaDB version 10.1+
    HTTPS support

We recommend Apache and NGINX as the most robust servers with the most features for running WordPress, but any server that supports PHP and MySQL will do.

Note: WordPress also works in legacy environments with PHP 5.6.20+ and MySQL 5.0+, but these versions have reached official end-of-life and as such may expose your site to security vulnerabilities.
```

Idzie zbyt łatwo? 

Czyżby [honeypot](https://en.wikipedia.org/wiki/Honeypot_(computing))?!

Nawet najmniejsze [numery wersji mają znaczenie](https://en.wikipedia.org/wiki/Software_versioning#Degree_of_compatibility), a [dobry changelog](https://www.apachelounge.com/Changelog-2.4.html) powie co załatano.

Jeśli nie możesz ustalić aktualnej wersji to przyjmij roboczo, że masz przed sobą coś odrobinę starszego. (Może admin jest zawalony robotą, a może są zależności, którch nie może zerwać i dlatego wersji nie podniósł).

### Wnioski

* przydatnym jest ustalić w co w ogóle celujemy,
* im dokładniejsze rozpoznanie tym więcej oczywistych furtek,
* [obscurity](https://en.wikipedia.org/wiki/Security_through_obscurity) to nie security, ale nie zaszkodzi (no bi niby dlaczego komuś cokolwiek ułatwiać).

## Gdzie i jak bić, żeby bolało?

Dobrze jest być jakiś czas [w development i w operations](https://en.wikipedia.org/wiki/DevOps), żeby dowiedzieć się gdzie najłatwiej o potknięcia lub którędy prowadzą skróty.

Możliwe, że cel zawiera otwarte komponenty. Same w sobie są one pewnie dość twarde, ale sposób ich użycia wcale poprawny być nie musi.

Nie trzeba czytać całego kodu, ale trzeba potrafić go sprawnie przeszukiwać i dlatego kochamy [`grep`](https://man7.org/linux/man-pages/man1/grep.1.html) i [regex-y](https://en.wikipedia.org/wiki/Regular_expression):
```
grep -P 'validate.+cookie' -l -R .
```

## Z wiedzy powszechnej

> (...) są takie rzeczy że nikt nie zaprzeczy; po co tu się głowić? (...)

Wszystko co wychodzi do klienta może wrócić od niego zmodyfikowane. Do sprawdzenia integralności przydają się tzw. [funkcje jednokierunkowe](https://en.wikipedia.org/wiki/One-way_function).

Krypto to sztuka sama sobie. Reguła kciuka - jak sie nie znasz to nie udawaj; weź coś gotowego. Każdy framework ma coś na pokładzie np. [do werfikacji integralności](https://en.wikipedia.org/wiki/HMAC).

Jednak znajomość mechanizmu nie może być wystarczająca do generowania legalnych treści, więc zawsze pamiętamy [o (unikatowej!) soli](https://en.wikipedia.org/wiki/Salt_(cryptography)).

> (...) wszyscy zgadzają się ze sobą, a będzie nadal tak jak jest (...)

cytaty (oczywiście) z "Co ty tutaj robisz?" Elektryczne Gitary

[Percent encoding](https://en.wikipedia.org/wiki/Percent-encoding) czasami zwą [url encoding](https://www.w3schools.com/tags/ref_urlencode.ASP).

PS. Kodowanie (encoding) to nie szyfrowanie (encryption).